# Run Hugging Face `codellama/CodeLlama-13b-hf` autoregressive sampling on Inf2 & Trn1

In this example we compile and deploy the Hugging Face [codellama/CodeLlama-13b-hf](https://huggingface.co/codellama/CodeLlama-13b-hf) model for tensor parallel inference on Neuron using the `transformers-neuronx` package. We use a sequence length of 16k.

The example has the following main sections:
1. Set up the Jupyter Notebook
1. Install dependencies
1. Download the model
1. Construct the model|
1. Split the model `state_dict` into multiple files
1. Perform autoregressive sampling using tensor parallelism

This Jupyter Notebook can be run on an Inf2 instance (`inf2.48xlarge`) or Trn1 instance (`trn1.32xlarge`).

## Set up the Jupyter Notebook

The following steps set up Jupyter Notebook and launch this tutorial:
1. Clone the [AWS Neuron Samples](https://github.com/aws-neuron/aws-neuron-samples) repo to your instance using
```
git clone https://github.com/aws-neuron/aws-neuron-samples.git
```
2. Navigate to the `transformers-neuronx` inference samples folder
```
cd aws-neuron-samples/torch-neuronx/transformers-neuronx/inference
```
3. Follow the instructions in [Jupyter Notebook QuickStart](https://awsdocs-neuron.readthedocs-hosted.com/en/latest/general/setup/notebook/setup-jupyter-notebook-steps-troubleshooting.html) to run Jupyter Notebook on your instance.
4. Locate this tutorial in your Jupyter Notebook session (`codellama-13b-16k-sampling.ipynb`) and launch it. Follow the rest of the instructions in this tutorial. 

## Install Dependencies
This tutorial requires the following pip packages:

 - `torch-neuronx`
 - `neuronx-cc`
 - `sentencepiece`
 - `transformers`
 - `transformers-neuronx`


Most of these packages will be installed when configuring your environment using the [torch-neuronx inference setup guide](https://awsdocs-neuron.readthedocs-hosted.com/en/latest/general/setup/torch-neuronx.html#setup-torch-neuronx). The additional dependencies must be installed here:

In [ ]:
!pip install transformers-neuronx sentencepiece

## Download and construct the model

We download and construct the model checkpoint using the `from_pretrained` function simply using the Huggingface model name.


## Construct the model

After downloading the model and converting it to the Hugging Face format we construct the model

In [ ]:
from transformers import LlamaForCausalLM

model = LlamaForCausalLM.from_pretrained('codellama/CodeLlama-13b-hf', low_cpu_mem_usage=True)

## Split the model state_dict into multiple files

For the sake of reducing host memory usage, it is recommended to save the model `state_dict` as
multiple files, as opposed to one monolithic file given by `torch.save`. This "split-format"
`state_dict` can be created using the `save_pretrained_split` function. With this checkpoint format,
the Neuron model loader can load parameters to the Neuron device high-bandwidth memory (HBM) directly
by keeping at most one layer of model parameters in the CPU main memory.

In [ ]:
from transformers_neuronx.module import save_pretrained_split

save_pretrained_split(model, './CodeLlama-13b-split')

## Perform autoregressive sampling using tensor parallelism

Now we have all of the necessary files for running `codellama/CodeLlama-13b-hf` autoregressive sampling. 

The memory required to host any model can be computed with:
```
total memory = bytes per parameter * number of parameters
```
When using `float16` casted weights for a 13 billion parameter model, this works out to `2 * 13B` or ~26GB of weights. Each NeuronCore has 16GB of memory which means that a 26GB model cannot fit on a single NeuronCore. In reality, the total space required is often greater than just the number of parameters due to caching attention layer projections (KV caching). This caching mechanism grows memory allocations linearly with sequence length and batch size.

To get very large language models to fit on Inf2 & Trn1, tensor parallelism is used to split weights, data, and compute across multiple NeuronCores. The number of NeuronCores that the weights are split across can be controlled by setting the `tp_degree` parameter. This parallelism degree must be chosen to ensure that the memory usage per NeuronCore will be less than the physical 16GB limit. When configuring tensor parallelism, the memory per NeuronCore can be computed with:

```
memory per core = (bytes per parameter * number of parameters) / tp_degree
```

This can be used to compute the minimum instance sizing by ensuring that the value selected for `tp_degree` results in less than 16GB allocated per NeuronCore.

Note that increasing the `tp_degree` beyond the minimum requirement almost always results in a faster model. Increasing the tensor parallelism degree improves memory bandwidth which improves model performance. To optimize performance it's recommended to use the highest tensor parallelism degree that is supported by the instance. In this sample we use tensor parallelism degree 32 to optimize performance on `trn1.32xlarge`, but this should be changed to 24 if you are using a `inf2.48xlarge`. 

We will use the Neuron `LlamaForSampling` class to implement tensor parallelism for the CodeLlama model. We supply the n_positions as 16384 and context length estimates to precompile various possible prompt lengths. Tensor parallelism is enabled through the argument `tp_degree=32`. We enable `float16` casting with the `amp='f16'` flag. The model computational graph is compiled by `neuronx-cc` for optimized inference on Neuron.

In [ ]:
import time
import torch
from transformers import AutoTokenizer
from transformers_neuronx import LlamaForSampling
import requests
import os
# Compiler flag -O1 is a workaround for “Too many instructions after unroll” in SDK 2.14
# It can also help with faster compilation
os.environ['NEURON_CC_FLAGS'] = '-O1'

# we supply a list of context_length_estimate that allows us to get low context encoding latency even for 
# larger prompts than 8192 (largest context_length_estimate by default is 1/2 of n_positions input)
context_length_estimate = [64, 128, 256, 512, 1024, 2048, 4096, 8192, 12288, 16384]

# load codellama/CodeLlama-13b-hf to the NeuronCores with 32-way tensor parallelism and run compilation
# we pass n_positions as 16384 so that the model supports 16k sequence length, and pass the context_length_estimate
# list generated above.
neuron_model = LlamaForSampling.from_pretrained('./CodeLlama-13b-split', n_positions=16384, \
                                                context_length_estimate=context_length_estimate, \
                                                batch_size=1, tp_degree=32, amp='f16')
neuron_model.to_neuron()

# construct a tokenizer and encode prompt text (prompt is loaded from a library and appended with instruction to write a function)
tokenizer = AutoTokenizer.from_pretrained('codellama/CodeLlama-13b-hf')
prompt = requests.get("https://raw.githubusercontent.com/kedartatwawadi/stanford_compression_library/e2fca703ac812331a277644ecc4ae5cfef160ab3/scl/compressors/lz77_sliding_window.py").text
prompt += "\n\n# Function to load binary data from user-provided file and compress it with LZ77 and write output to file\n"
input_ids = tokenizer.encode(prompt, return_tensors="pt") 
num_input_tokens = len(input_ids[0]) # over 11k tokens
print(f"num_input_tokens: {num_input_tokens}")

# run inference with top-k sampling
with torch.inference_mode():
    start = time.time()
    generated_sequences = neuron_model.sample(input_ids, sequence_length=16384, top_k=50)
    elapsed = time.time() - start

# display the new generated tokens
generated_sequences = [tokenizer.decode(seq[num_input_tokens:]) for seq in generated_sequences]
print(f'generated sequence {generated_sequences[0]} in {elapsed} seconds')

## Save and load the compiled model

The ```save``` and ```load``` functions can be used to save and load compiled model artifacts respectively. Loading compiled model artifacts from a provided directory will avoid model recompilation.

In [ ]:
neuron_model.save('./neuron_artifacts') # can be copied and used on a different neuron instance
del neuron_model
neuron_model = LlamaForSampling.from_pretrained('./CodeLlama-13b-split', n_positions=16384, \
                                                context_length_estimate=context_length_estimate, \
                                                batch_size=1, tp_degree=32, amp='f16')
neuron_model.load('neuron_artifacts') # Load the compiled Neuron artifacts
neuron_model.to_neuron() # will skip compile

with torch.inference_mode():
    start = time.time()
    generated_sequences = neuron_model.sample(input_ids, sequence_length=16384, top_k=50)
    elapsed = time.time() - start

generated_sequences = [tokenizer.decode(seq[num_input_tokens:]) for seq in generated_sequences]
print(f'generated sequence {generated_sequences[0]} in {elapsed} seconds')